# Traducción de Carácteres Alfanuméricos a Código Morse y viceversa


**Asignatura: Autómatas y Lenguajes Formales**

**Profesor: Gustavo Garzón**



**Estudiantes:**

- Carvajal Esparza Juan Sebastian, 2143162

- Cerinza Zaraza Juan Pablo, 2190081

- Gualdron Hurtado Yesid Romario, 2190052

    


**Escuela de Ingeniería de Sistemas e Informática**   

_Septiembre de 2020_

In [1]:

from project_automatons import automaton_library
import regex as rg


## Introducción

Para la realización de este proyecto, se hizo uso de dos Máquinas de Turing, inplementadas con la libreria  [automatalib](https://pypi.python.org/pypi/automata-lib/1.0.0.post4) y también mediante la herramienta [JFLAP](http://www.jflap.org/). Cada máquina de Turing tiene  una función diferente, la primera traduce el código morse ingresado en sus correspondientes carácteres alfanumericos, la segunda, a partir del texto ingresado, encuentra su equivalente en código morse. Es necesario tener en cuenta la siguiente imagen donde se muestran las reglas del código morse:


<img src="files/Morse.jpeg" />

## Definición Formal

### Primera Maquina de Turing (Morse $\rightarrow$ Carácteres Alfanuméricos)

Como trabajo previo a la eleboración de esta máquina de Turing, se realizó un Autómata Finito Determinista, en JFLAP, el cual "traducía" de código morse a su carácter alfanumérico correspondiente, cabe recalcar que, por orden y como convención para este Árbol de Busqueda Binaria las ramas derechas de cada nodo leen los guiones y las izquierdas los puntos, pero este automata en forma de árbol solo aceptaba cadenas de máximo 5 caracteres entre puntos y rayas, como se muestra a continuación:
<img src="files/arbol-afd.png" />

Luego, como solución para la traducción de varios caracteres se decidió implementar este árbol como una máquina de Turing, pero debido a la cantidad de caracteres alfanumericos y al proceso de traducción como tal, se mostrará un modelo base para la explicación de dicha máquina de Turing:
<img src="files/modelobasico-mt.jpeg" />

MT1 = (Q, q0, F, Σ, Γ,B, δ)

Q: { q000, qE , qT , Qt , q099 , Qe , q101 }

q0 : q000
F: { q101 }
Σ: { . , - ,    , }
Γ: { . , - ,  , # , a , b , c , d , e , f , g , h , i , j , k , l , m , n , o , p , q , r , s , t , u , v , w,  x , y , z , 0 , 1 , 2 , 3 , 4 , 5 , 6 , 7 , 8 , 9 }
B : #
δ : Q × Γ 
    δ( q00 , - ) = ( qT , # , R )        δ( q099 , - ) = ( q099 , - , L )
    δ( q00 , . ) = ( qE , # , R )        δ( q099 , . ) = ( q099 , . , L )
    δ( qT  ,     ) = ( Qt , # , R )      δ(q099 , # ) = ( q000 , # , R )
    δ( qE ,      ) = ( Qe , # , R )      δ( q100 , t ) = ( q100 , t , L )
    δ( Qe , t  ) = ( Qe , t , R )        δ(q100 , e ) = ( q100 , e , L )
    δ( Qe , e ) = ( Qe , e , R )         δ( q100 ,   ) = ( q100 ,   , L )
    δ( Qe , -  ) = ( Qe , - , R )        δ( q100 , # ) = ( q101 , # , S )
    δ( Qe , . ) =  ( Qe , . , R )        δ( Qt , t ) = ( Qt , t , R )
    δ( Qe ,   ) =  ( Qe ,   , R )        δ( Qt , e ) = ( Qt , e , R )
    δ( Qe , # ) = ( q100 , e , L )       δ( Qt , - ) = ( Qt , - , R )
    δ( q00 , . ) = ( q099 , . , L )      δ( Qt , . ) = ( Qt , . , R )
    δ( q00 , - ) = ( q099 , - , L )      δ( Qt ,   ) = ( Qt ,   , R )
    δ( q099 ,  )  = ( q099 ,   , L )     δ( Qt , # ) = ( q100 , t , L)

Se hará una descripción de los diferentes tipos de estados presentes en la máquina de Turing:
- q000: Es el estado inicial y tiene transiciones para leer un punto o un guion
- qE o qT: Llegar a un estado de este tipo indica que puede ser ese caracter alfanumerico el equivalente al codigo morse leido
- Qe o Qt: Luego de leer un espacio y llegar a este estado, se confirma que este caracter es el equivalente a la seccion del codigo morse ingresado
- q100: En este estado llegan todas las transiciones luego de que se haya guardado en el final de cinta el caracter correspondiente a la traduccion, por lo cual este procede a ignorar hacia la izquieda todos los caracteres alfanumericos de la cinta
- q101: Si despues de ignorar los caracteres alfanumericos no se encuentra ni un punto ni una raya a la izquierda de ellos, se entiende que la traduccion ha finalizado y llega a este estado de aceptación
- q99: Con este estado se ignoran hacia la izquierda los posibles puntos y rayas que tenga la cinta en ese momento, al final, cuando encuentre un espacio vacío, se desplaza hacia el estado inicial para traducir otro caracter alfanumerico

Una combinación de este modelo base y el árbol de busqueda binaria, es la representación de la máquina de turing trabajada, por la cantidad de estados y transiciones para ignorar caracteres en el momento de la traduccion, un diagrama de esta maquina no sería visualmente entendible. 




---

<h1>Implementación</h1>

<h3>Traductor morse-latino</h3>

Para traducir de morse al abecedario latino, incluyendo números.
Es necesario asegurarse que después de cada letra, haya un espacio (" "), así también que entre cada palabra, exista un "/".

<h4>Ejemplo</h4>

Escriba el código morse a traducir: <br>
<div style="border-style: solid; width: 280px; height: 45px; text-align: center; border-width: 1px;">
<h3>.... . .-.. .-.. --- /.-- --- .-. .-.. -.. </h3>
</div>
<br>
Output: <br>
hello world

El siguiente bloque de código realiza:
- Descompone la frase en palabras y las traduce una por una.
- Traduce una por una las palabras, mostrando un mensaje del error si surge uno.
- Cada palabra traducida, es adicionada a una lista de strings, que al final se junta en un string y lo imprime.
- Si se encuentra un error, muestra la palabra en donde residió el error y en su lugar pone "???"


In [2]:
# Word separation using /

decoded_morse = list()

automaton = automaton_library.morse_to_latin

morse_code = input("Escriba el código morse a traducir: ")

morse_code = rg.split('/', morse_code)

if morse_code[-1][-1] != ' ':
    fix = morse_code[-1] + ' '
    morse_code[-1] = fix

for morse_word in morse_code:
    try:
        word = automaton.input_tape(morse_word)
    except:
        print('La palabra "' + morse_word + '" está errónea. Omitiendo palabra....')
        word = '???'
        
    decoded_morse.append(word)    

decoded_morse = ' '.join(decoded_morse)

print("La traduccion es: " + decoded_morse)


#tape = automaton_library.morse_to_latin.input_tape('. -- ----- .---- ..--- ...-- ....- ..... -.... --... ---.. ----. ')
#print(tape)


Escriba el código morse a traducir:  .... . .-.. .-.. --- /.-- --- .-. .-.. -..


La traduccion es: hello world


In [3]:
coded_morse = list()
morse_word = list()

automaton = automaton_library.latin_to_morse

morse_code = input("Escriba el texto a traducir: ")

morse_code = rg.split(' ', morse_code)


for word in morse_code:
    try:
        for letter in word:
            morse_word.append(automaton.input_tape(letter))
        coded_morse.append(' '.join(morse_word))
        morse_word = list()
    except:
        print("Ocurrió un error, por favor asegúrese que está ingresando caracteres válidos")
        
coded_morse = ' /'.join(coded_morse)

print(coded_morse)
            

Escriba el texto a traducir:  hello world


.... . .-.. .-.. --- /.-- --- .-. .-.. -..


In [4]:

from pydub import AudioSegment
from pydub.playback import play

dash = AudioSegment.from_ogg("files/dash.ogg")
dot = AudioSegment.from_ogg("files/dot.ogg")

dot_silence = AudioSegment.silent(duration=414)
sound = dot_silence

for char in coded_morse:
    if char == '-':
        sound += dash
    if char == '.':
        sound += dot
    if char == ' ':
        sound += dot_silence * 3
    if char == '/':
        sound +=  dot_silence * 5
    
    
play(sound)
    

sudo apt-get install ffmpeg libavcodec-extra
pip install pydub ffmpeg ffprobe regex

---

[1] Implementación de una Máquina de Turing universal: https://rosettacode.org/wiki/Universal_Turing_machine

[2] Libreria en Python para máquinas de Turing: https://pypi.python.org/pypi/turing_machine